# Training 

In [2]:
import os
os.environ["KERAS_BACKEND"] = "torch"

import keras_core
import matplotlib.pyplot as plt

Using PyTorch backend.


In [3]:
train_ds = keras_core.utils.image_dataset_from_directory(
    directory='dataset/training_data',
    color_mode='rgb',
    label_mode='categorical',
    batch_size=16,
    image_size=(256, 256))


validation_ds, testing_ds = keras_core.utils.image_dataset_from_directory(
    directory='dataset/validation_data',
    label_mode='categorical',
    color_mode='rgb',
    validation_split=0.5,
    subset='both',
    seed=123,
    batch_size=16,
    image_size=(256, 256))


print(f"Number of training samples: {train_ds.cardinality()}")
print(f"Number of validation samples: {validation_ds.cardinality()}")
print(f"Number of testing samples: {testing_ds.cardinality()}")

Found 5712 files belonging to 4 classes.
Found 1311 files belonging to 4 classes.
Using 656 files for training.
Using 655 files for validation.
Number of training samples: 357
Number of validation samples: 41
Number of testing samples: 41


In [12]:
base_model = keras_core.applications.efficientnet.EfficientNetB3(
    weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_shape=(256, 256, 3),
    include_top=False,
    pooling="max",
)

model = keras_core.Sequential(
    [   
        keras_core.layers.Input(shape=(256, 256, 3)),
        base_model,
        keras_core.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001),
        keras_core.layers.Dense(
            256,
            kernel_regularizer=keras_core.regularizers.l2(0.016),
            activity_regularizer=keras_core.regularizers.l1(0.006),
            bias_regularizer=keras_core.regularizers.l1(0.006),
            activation="relu",
        ),
        keras_core.layers.Dropout(rate=0.45, seed=123),
        keras_core.layers.Dense(4, activation="softmax", ),
    ]
)
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape              ┃    Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━┩
│ efficientnetb3 (Functional)     │ (None, 1536)              │ 10,783,535 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ batch_normalization_2           │ (None, 1536)              │      6,144 │
│ (BatchNormalization)            │                           │            │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense_4 (Dense)                 │ (None, 256)               │    393,472 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dropout_2 (Dropout)             │ (None, 256)               │          0 │
├─────────────────────────────────┼───────────────────────────┼────────────┤
│ dense_5 (Dense)                 │ (None, 4)                 │      1,028 │
└─────────────────────────────────┴───────────────────────────┴────────────┘

 Total params: 11,184,179 (42.66 MB)

 Trainable params: 11,093,804 (42.32 MB)

 Non-trainable params: 90,375 (353.03 KB)

In [25]:
early_stop = keras_core.callbacks.EarlyStopping(
    monitor="val_loss", patience=5, restore_best_weights=True, mode="min"
)

model.compile(
    keras_core.optimizers.Adamax(learning_rate=0.001),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)

epochs = 20
model.fit(train_ds, epochs=epochs, validation_data=validation_ds, batch_size=16)

model.evaluate(testing_ds)

Epoch 1/20
  1/357 ━━━━━━━━━━━━━━━━━━━━ 1:51:50 19s/step - accuracy: 0.1875 - loss: 21.8647

KeyboardInterrupt: 

In [15]:
model.load_weights("weight.weights.h5")

model.compile(
    keras_core.optimizers.Adamax(learning_rate=0.001),
    loss="categorical_crossentropy",
    metrics=["accuracy"],
)

In [16]:
model.evaluate(testing_ds)

 5/41 ━━━━━━━━━━━━━━━━━━━━ 3:20 6s/step - accuracy: 0.3533 - loss: 5.6840      

KeyboardInterrupt: 